In [10]:
%load_ext autoreload
%autoreload 2

from utils.visualize_nii import visualize_mri_sample
from data.bonn_data import BonnMRIClassificationDataset
import torch
import matplotlib.pyplot as plt

data = BonnMRIClassificationDataset('./Dataset')

sample = data[0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Using deivce:', device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using deivce: cuda


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from models.classifier import FCDDetector

model = FCDDetector()

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss with logits

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(data))
val_size = int(0.15 * len(data))
test_size = len(data) - train_size - val_size
train_data, val_data, test_data = random_split(data, [train_size, val_size, test_size])

# Create DataLoaders for train and validation datasets
batch_size = 16  # Define batch size
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Initialize lists to store loss information
train_losses = []
val_losses = []

# Training loop with validation
num_epochs = 10  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        print(batch)
        inputs = batch['FLAIR']  # Input MRI data
        labels = batch['label'].float()  # Corresponding labels (convert to float for BCEWithLogitsLoss)

        # Move data to GPU if available
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
            model = model.cuda()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training complete.")

FileNotFoundError: No such file or no access: 'Dataset/sub-00120/anat/sub-00120_acq-T2sel_FLAIR.nii.gz'